In [2]:
import pandas as pd
import numpy as np
import json
import re


pd.set_option('display.max_columns', None)
print("Libraries loaded successfully.")

Libraries loaded successfully.


In [4]:
df_orders = pd.read_csv('orders.csv')
df_users = pd.read_json('users.json')

def parse_restaurant_sql(file_path):
    data = []
    pattern = re.compile(r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([0-9.]+)\)")

    with open(file_path, 'r') as file:
        for line in file:
            if line.strip().startswith("INSERT INTO"):
                match = pattern.search(line)
                if match:
                    data.append(match.groups())

    return pd.DataFrame(data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

df_restaurants = parse_restaurant_sql('restaurants.sql')

df_restaurants['restaurant_id'] = df_restaurants['restaurant_id'].astype(int)
df_restaurants['rating'] = df_restaurants['rating'].astype(float)

In [6]:
df_main = df_orders.merge(df_users, on='user_id', how='left') \
                   .merge(df_restaurants, on='restaurant_id', how='left', suffixes=('', '_sql'))

df_main['order_date'] = pd.to_datetime(df_main['order_date'], dayfirst=True)


In [7]:
gold_data = df_main.query("membership == 'Gold'")
top_gold_city = gold_data.groupby('city')['total_amount'].sum().nlargest(1).index[0]
print(f"Top Gold City: {top_gold_city}")

cuisine_avg = df_main.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print(f"Top Avg Value Cuisine: {cuisine_avg.index[0]}")


Top Gold City: Chennai
Top Avg Value Cuisine: Mexican


In [8]:
user_lifetime_value = df_main.groupby('user_id')['total_amount'].sum()
high_rollers_count = user_lifetime_value[user_lifetime_value > 1000].count()
print(f"Users spending > 1000: {high_rollers_count}")

rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
bin_labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

df_main['rating_category'] = pd.cut(df_main['rating'], bins=rating_bins, labels=bin_labels, include_lowest=True)
top_rating_tier = df_main.groupby('rating_category', observed=False)['total_amount'].sum().idxmax()
print(f"Highest Revenue Rating Tier: {top_rating_tier}")

Users spending > 1000: 2544
Highest Revenue Rating Tier: 4.6 – 5.0


In [9]:
best_gold_city_avg = gold_data.groupby('city')['total_amount'].mean().idxmax()
print(f"Best City for Gold AOV: {best_gold_city_avg}")

cuisine_diversity = df_main.groupby('cuisine')['restaurant_id'].nunique()
least_diverse_cuisine = cuisine_diversity.idxmin()
print(f"Cuisine with fewest restaurants: {least_diverse_cuisine}")

total_orders = len(df_main)
gold_orders_count = len(gold_data)
gold_percentage = (gold_orders_count / total_orders) * 100
print(f"Gold Member Order %: {int(round(gold_percentage))}%")

Best City for Gold AOV: Chennai
Cuisine with fewest restaurants: Chinese
Gold Member Order %: 50%


In [10]:
candidates = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

target_df = df_main[df_main['restaurant_name'].isin(candidates)]
stats = target_df.groupby('restaurant_name').agg(
    order_count=('order_id', 'count'),
    avg_val=('total_amount', 'mean')
)

result_restaurant = stats[stats['order_count'] < 20]['avg_val'].idxmax()
print(f"Target Restaurant: {result_restaurant}")

Target Restaurant: Ruchi Foods Chinese


In [11]:
matrix = df_main.pivot_table(index='membership', columns='cuisine', values='total_amount', aggfunc='sum')

pairs_to_check = [('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]
results = {f"{m} + {c}": matrix.loc[m, c] for m, c in pairs_to_check}
best_combo = max(results, key=results.get)
print(f"Best Combo: {best_combo}")

best_quarter = df_main.groupby(df_main['order_date'].dt.quarter)['total_amount'].sum().idxmax()
print(f"Best Quarter: Q{best_quarter}")

Best Combo: Gold + Italian
Best Quarter: Q3


In [12]:
print(f"Distinct Users: {df_main['user_id'].nunique()}")
print(f"Total Gold Orders: {len(gold_data)}")
print(f"Hyderabad Total Revenue: {round(df_main[df_main['city'] == 'Hyderabad']['total_amount'].sum())}")
print(f"Gold Avg Order Value: {round(gold_data['total_amount'].mean(), 2)}")
print(f"High Rated Orders (>= 4.5): {len(df_main[df_main['rating'] >= 4.5])}")

top_city_name = gold_data.groupby('city')['total_amount'].sum().idxmax()
count_in_top = len(gold_data[gold_data['city'] == top_city_name])
print(f"Gold Orders in {top_city_name}: {count_in_top}")

Distinct Users: 2883
Total Gold Orders: 4987
Hyderabad Total Revenue: 1889367
Gold Avg Order Value: 797.15
High Rated Orders (>= 4.5): 3374
Gold Orders in Chennai: 1337
